# 02. Preprocesarea datelor

In [1]:
! pip install pyodbc
! pip install sqlalchemy

In [2]:
import pandas as pd
import json
import pyodbc
import sqlalchemy as db
import urllib

exec(open('azure_credentials.py').read())

In [3]:
username

'aid2023adminlog'

## Load tables

In [4]:
# Load csv schoolClasses
schoolClasses_Team5 = pd.read_csv('schoolClasses.csv')
schoolClasses_Team5.head(3)

,id,idSchool,idBuilding,buildingName,code,name,roomType,surface,floor,seatsNo,computersNumber
0,184621,11269910,34136,Gradinita Gosen Arad,C1,Clasa 1,Sală de clasă,72.01,1,30.0,NaN
1,185155,11269910,34136,Gradinita Gosen Arad,S1,Sala de sport,Sală de sport,79.04,2,NaN,NaN
2,184858,11269910,34136,Gradinita Gosen Arad,C2,Clasa 2,Sală de clasă,72.01,1,30.0,NaN


In [5]:
# Load csv schoolTransport
schoolTransport_Team5 = pd.read_csv('schoolTransport.csv')
schoolTransport_Team5.head(3)

,idSchool,seatsNo,acquisitionYear
0,11264262,9,2008
1,11263948,28,2020
2,11263948,17,2015


In [6]:
# load csv evnat_2021
evnat_2021_Team5 = pd.read_csv('evnat_2021.csv')
evnat_2021_Team5.head(3)

,idSchool,Media
0,11270625,9.73
1,11261034,9.71
2,11269669,9.68


In [7]:
# Load csv details
details_Team5 = pd.read_csv('details.csv')
details_Team5.head(3)

,idSchool,internalIdSchool,idParentSchool,schoolYearDescription,code,siruesCode,longName,shortName,schoolType,statut,...,street,streetNumber,postalCode,phoneNumber,faxNumber,email,idSchoolYear.code,Web_Link,Fb_Link,Adress
0,11269910,10131270,NaN,Anul şcolar 2018-2019,261200154,23054595,"ASOCIAŢIA CREŞTINĂ DE CARITATE ""GOSEN"" ARAD/ G...","GRĂDINIŢA ""GOSEN"" ARAD",Unitate de învățământ,Cu personalitate juridică,...,AUREL VLAICU,121-125,310365.0,257272477.0,257272477.0,gradinitagosen@yahoo.com,2018-2019,NaN,NaN,"121-125 AUREL VLAICU, ARAD, judetul ARAD, 3103..."
1,11256876,11192270,NaN,Anul şcolar 2018-2019,1061206943,NaN,ASOCIAȚIA PENTRU PROMOVAREA VALORILOR AUTENTIC...,ASOCIAȚIA VALORILOR BZ,Unitate de învățământ,Cu personalitate juridică,...,TOAMNEI,4,125300.0,762724000.0,238563970.0,secretariat@cnpaulescu.ro,2018-2019,NaN,NaN,"4 TOAMNEI, RÂMNICU SĂRAT, judetul BUZĂU, 12530..."
2,11256865,11192168,NaN,Anul şcolar 2018-2019,1061206875,NaN,ASOCIAȚIA UMANITARĂ EDUSTAR BUZĂU,ASOCIATIA EDUSTAR,Unitate de învățământ,Cu personalitate juridică,...,STR. MIORITEI,14,120010.0,773826330.0,238725913.0,luminitamusat2006@yahoo.com,2018-2019,NaN,NaN,"14 STR. MIORITEI, BUZĂU, judetul BUZĂU, 120010..."


## Connection details

In [8]:
database_name = "AID2023"
driver = '{ODBC Driver 18 for SQL Server}'
server_name = "aid-2023-srv.database.windows.net"

#### CONNECT TO AZURE DATABASE

In [9]:
# Create the server url
server = 'tcp:{server_name},1433'.format(server_name = server_name)

#create the full connection string <- make sure to write it as it is bellow - check your credentials to be the right ones

connection_string = 'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};charset=utf8'.format(
    driver = driver,
    server=server,
    database=database_name,
    username=username,
    password=password
)                              

cnxn: pyodbc.Connection = pyodbc.connect(connection_string)
cursor: pyodbc.Cursor = cnxn.cursor() 

#execute a query with the cursor to check if the connection is maade
print("Connection is open") if cursor.execute("SELECT 1") else print(f"Connection is closed: {e}")

Connection is open


### USE SQL ALCHEMY TO LOAD THE DATAFRAMES

In [10]:
#obtain the parameters from the pyodbc connection string
params = urllib.parse.quote_plus(connection_string)
#create the engine
sqlalchemy_engine = db.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### UPLOAD TO SQL

In [11]:
#insert the values form details df into sql table
schoolClasses_Team5.to_sql("SchoolClasses_Team5", sqlalchemy_engine, if_exists='replace', index=False)
schoolTransport_Team5.to_sql("SchoolTransport_Team5", sqlalchemy_engine, if_exists='replace', index=False)
evnat_2021_Team5.to_sql("Evnat_2021_Team5", sqlalchemy_engine, if_exists='replace', index=False)

PendingRollbackError: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b)

In [12]:
details_Team5.to_sql("Details_Team5", sqlalchemy_engine, if_exists='replace', index=False)

PendingRollbackError: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b)